In [2]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker
import matplotlib.pyplot as plt
import torch
import sys
sys.path.append('..')
from functions import *

/local/data/scvi-tools/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/local/data/scvi-tools/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  dispatcher = jit(_target='npyufunc',
/local/data/anaconda3/envs/BT/lib/python3.11/site-pac

In [4]:
adata = anndata.read_h5ad(filename="../data/adataMou4.h5ad")
adata

AnnData object with n_obs × n_vars = 97861 × 4000
    obs: 'batch', 'chemistry', 'data_type', 'dpt_pseudotime', 'final_annotation', 'mt_frac', 'n_counts', 'n_genes', 'size_factors', 'species', 'study', 'tissue'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'pca'
    obsm: 'Scanorama', 'Unintegrated', 'X_pca'
    varm: 'PCs'
    layers: 'counts'

In [5]:
sdnormalAdata, vaeSD = trainModelBenchmark(adata.copy(), "sdnormal", log=True, logname="sdMouBench",save="models/MBM_StandardNormal")

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/sdMouBench
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [36:45<00:00,  5.61s/it, v_num=0, train_loss_step=1.32e+3, train_loss_epoch=1.67e+3]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [36:45<00:00,  5.51s/it, v_num=0, train_loss_step=1.32e+3, train_loss_epoch=1.67e+3]


In [6]:
mogAdata, vaeMG = trainModelBenchmark(adata.copy(), "mixofgaus",log=True,logname="mogMouBench",save="models/MBM_MixtureOfGaussians")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/mogMouBench
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [40:40<00:00,  5.31s/it, v_num=0, train_loss_step=1.6e+3, train_loss_epoch=1.66e+3] 

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [40:40<00:00,  6.10s/it, v_num=0, train_loss_step=1.6e+3, train_loss_epoch=1.66e+3]


In [7]:
vampAdata, vaeVP = trainModelBenchmark(adata.copy(), "vamp",log=True,logname="vampMouBench",save="models/MBM_Vamp")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/vampMouBench
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/400:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch 400/400: 100%|██████████| 400/400 [39:46<00:00,  6.41s/it, v_num=0, train_loss_step=1.28e+3, train_loss_epoch=1.66e+3]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [39:46<00:00,  5.97s/it, v_num=0, train_loss_step=1.28e+3, train_loss_epoch=1.66e+3]


In [8]:
flowAdata, vaeNF = trainModelBenchmark(adata.copy(), 'normalflow', log=True, logname="flowMouBench",save="models/MBM_NormalFlow")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/flowMouBench
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [1:08:43<00:00, 10.56s/it, v_num=0, train_loss_step=2.87e+3, train_loss_epoch=1.66e+3]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [1:08:43<00:00, 10.31s/it, v_num=0, train_loss_step=2.87e+3, train_loss_epoch=1.66e+3]


In [ ]:
vaeSD = scvi.model.SCVI.load("models/MBM_StandardNormal")
vaeMG = scvi.model.SCVI.load("models/MBM_MixtureOfGaussians")
vaeVP = scvi.model.SCVI.load("models/MBM_Vamp")
vaeNF = scvi.model.SCVI.load("models/MBM_NormalFlow")

In [9]:
allAdata = adata.copy()
allAdata.obsm["scVISD"] = vaeSD.get_latent_representation()
allAdata.obsm["scVIMG"] = vaeMG.get_latent_representation()
allAdata.obsm["scVIVAMP"] = vaeVP.get_latent_representation()
allAdata.obsm["scVINF"] = vaeNF.get_latent_representation()

In [10]:
plotBenchmarkResults(allAdata, keys=["Unintegrated","Scanorama", "scVISD", "scVIMG", "scVIVAMP", "scVINF"],label_key="final_annotation",save_dir="Benchmarks/")

Embeddings:   0%|          | 0/6 [00:00<?, ?it/s]INFO:root:isolated labels: no more than 1 batches per label
INFO:root:T cells: 0.39853960275650024


INFO     T cells consists of a single batch or is too small. Skip.                                                 


Embeddings:  17%|█▋        | 1/6 [02:37<13:07, 157.45s/it]INFO:root:isolated labels: no more than 1 batches per label
INFO:root:T cells: 0.49366109212860465


INFO     T cells consists of a single batch or is too small. Skip.                                                 


Embeddings:  33%|███▎      | 2/6 [27:07<1:01:57, 929.35s/it]INFO:root:isolated labels: no more than 1 batches per label
INFO:root:T cells: 0.396405354142189


...

Embeddings:  67%|██████▋   | 4/6 [31:32<13:13, 396.95s/it]INFO:root:isolated labels: no more than 1 batches per label
INFO:root:T cells: 0.3593131899833679


INFO     T cells consists of a single batch or is too small. Skip.                                                 


Embeddings:  83%|████████▎ | 5/6 [33:42<05:00, 300.81s/it]INFO:root:isolated labels: no more than 1 batches per label
INFO:root:T cells: 0.3737674355506897


INFO     T cells consists of a single batch or is too small. Skip.                                                 


Embeddings: 100%|██████████| 6/6 [35:58<00:00, 359.72s/it]


In [11]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Reusing TensorBoard on port 6006 (pid 135879), started 5:13:52 ago. (Use '!kill 135879' to kill it.)